### The purpose of this script is the following:

1. Pick the model epoch with the best performing kappa score
2. Use d4_transform for TTA which consists of a horizontal flip and 4 different 90 degree rotations. To aggregate the results, I selected the mean of the prediction scores
3. Compute the kappa score when using TTA and not TTA and compare the results

<br>

Results:
* TTA did improve the performance for the B3 model: 
    * QuadraticWeightedKappa (Non-TTA): 0.7974120604486354
    * QuadraticWeightedKappa (TTA): 0.8016524965905782


In [10]:
import torch
from torch import nn, optim
import os
import config
from torch.utils.data import DataLoader
from tqdm import tqdm
from sklearn.metrics import cohen_kappa_score
from efficientnet_pytorch import EfficientNet
from dataset import DRDataset
from torchvision.utils import save_image
from torchsummary import summary
from utils import (
    B3Config,
    B4Config,
    B5Config,
    load_checkpoint,
    save_checkpoint,
    check_accuracy,
    make_prediction,
    get_csv_for_blend,
)
from os import listdir
from os.path import isfile, join
import pandas as pd
import numpy as np
import ttach as tta

### Code to output predictions

In [ ]:

def make_prediction(model, loader, output_csv="submission.csv"):
    preds = []
    filenames = []
    model.eval()
    all_preds, all_labels = [], []

    for x, y, files in tqdm(loader):
        x = x.to(config.DEVICE)
        with torch.no_grad():
            predictions = model(x)
            # Convert MSE floats to integer predictions
            predictions[predictions < 0.5] = 0
            predictions[(predictions >= 0.5) & (predictions < 1.5)] = 1
            predictions[(predictions >= 1.5) & (predictions < 2.5)] = 2
            predictions[(predictions >= 2.5) & (predictions < 3.5)] = 3
            predictions[(predictions >= 3.5) & (predictions < 10000000)] = 4
            predictions = predictions.long().squeeze(1)
            preds.append(predictions.cpu().numpy())

            all_preds.append(predictions.detach().cpu().numpy())
            all_labels.append(y.detach().cpu().numpy())
            filenames += files

    df = pd.DataFrame({"image": filenames, "level": np.concatenate(preds, axis=0)})
    df.to_csv(output_csv, index=False)
    model.train()
    print("Done with predictions")
    return (np.concatenate(all_preds, axis=0, dtype=np.int64), np.concatenate(all_labels, axis=0, dtype=np.int64))

### Determine best model (for this code only pick 1 of the models to test TTA)

In [2]:

model_checkpoint_path = 'C:/GitHub/chrisnielsen-ophthalmology/diabetic-retinopathy-classification/DiabeticRetinopathy/models'
results_output_path = 'C:/GitHub/chrisnielsen-ophthalmology/diabetic-retinopathy-classification/DiabeticRetinopathy/results'
existing_result_files = [f for f in listdir(results_output_path) if isfile(join(results_output_path, f))]


model_config_dict = {'b3': B3Config(),
                     'b4': B4Config(),
                     'b5': B5Config()}


for model_type in ['b3','b4','b5']:
    print(model_type)
    model_config = model_config_dict[model_type]

    for ensemble_iter in range(model_config.number_of_ensemble_iters):
        model_output_log_name = model_type + '_' + str(ensemble_iter) + '.csv'
        if model_output_log_name in existing_result_files:
            results_df = pd.read_csv(results_output_path + '/' + model_output_log_name)
        else:
            continue
        
        best_performing_model_epoch = results_df.kappa.argmax()
        
        
        val_ds = DRDataset(
                images_folder="C:/Data/Kaggle EyePACS/test_images_resized_512/",
                path_to_csv="C:/Data/Kaggle EyePACS/test_public.csv",
                transform=model_config.val_transforms,
            )
        
        
        val_loader = DataLoader(
            val_ds,
            batch_size=model_config.batch_size,
            shuffle=False,
            num_workers=12, persistent_workers=True
        )
        
        
        
        loss_fn = nn.MSELoss()

        print('loading', model_config.model_name)
        model = EfficientNet.from_pretrained(model_config.model_name)
        model._fc = nn.Linear(model_config.fc_size, 1)
        model = model.to(config.DEVICE)
        optimizer = optim.Adam(model.parameters(), lr=config.LEARNING_RATE, weight_decay=config.WEIGHT_DECAY)
        scaler = torch.cuda.amp.GradScaler()


        print('load model')
        model_path = model_checkpoint_path + '/' + model_type + '_' + str(ensemble_iter) + '_' + str(best_performing_model_epoch) + ".pth.tar"
        load_checkpoint(torch.load(model_path), model, optimizer, config.LEARNING_RATE)
        

        break
    break


b3
loading efficientnet-b3
Loaded pretrained weights for efficientnet-b3
load model
=> Loading checkpoint


### Compute non-TTA predictions

In [13]:
preds, labels = make_prediction(model, val_loader, "predictions/submission_.csv")

100%|████████████████████████████████████████████████████████████████████████████████| 455/455 [01:08<00:00,  6.68it/s]

Done with predictions


### Compute TTA predictions

In [15]:
tta_model = tta.ClassificationTTAWrapper(model, tta.aliases.d4_transform())

In [16]:
preds_tta, labels_tta = make_prediction(tta_model, val_loader, "predictions/submission_2.csv")

100%|████████████████████████████████████████████████████████████████████████████████| 455/455 [07:17<00:00,  1.04it/s]

Done with predictions


### Compute weighted kappa results for non-TTA and TTA predictions

In [17]:
print(f"QuadraticWeightedKappa (Non-TTA): {cohen_kappa_score(labels, preds, weights='quadratic')}")
print(f"QuadraticWeightedKappa (TTA): {cohen_kappa_score(labels_tta, preds_tta, weights='quadratic')}")

QuadraticWeightedKappa (Non-TTA): 0.7974120604486354
QuadraticWeightedKappa (TTA): 0.8016524965905782
